In [34]:
#! pip install oracledb

username = "ora_shrutivs"
pw = "a55121776"

import oracledb
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
import numpy as np

dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
connection = oracledb.connect(user=username, password=pw, dsn=dsn)

In [35]:
def generate_df(lst, name, cols):
    """
    lst: empty list for each dataframe
    
    name: name of table in schema (str)
    
    cols: list of column names for table
    """
    cur = connection.cursor()
    cur.execute(f"select * from {name}")
    res = cur.fetchall()
    for row in res:
        lst.append(row)
    cur.close()
    lst_df = pd.DataFrame(lst, columns = cols)
    return lst_df

In [36]:
continent_df = generate_df([], "Continent", ["continentName", "code"])
country_df = generate_df([], "Country", ["name", "continentName", "ISO3"])
sea_level_df = generate_df([], "SeaLevel", ["SealevelYear", "continentName", "unit", "GMSL"])
ghg_emission_df = generate_df([], "GHGEmission", ["ghgYear","unit"])
temperature_df = generate_df([], "Temperature", ["year", "ghgYear","unit"])
industry_df = generate_df([], "Industry", ["industryName"])
emitted_df = generate_df([], "Emitted", ["continentName", "ghgYear", "co2concentration"])
produces_df = generate_df([], "Produces", ["ghgYear", "industryName",  "totalco2concentration"])
temp_change_df = generate_df([], "TempChange", ["countryName", "year", "ghgYear", "temperature"])


In [37]:
# Dictionary of dataframes
dfs = {
    'Produces': produces_df,
    'Continent': continent_df,
    'Country': country_df,
    'GHG Emission': ghg_emission_df,
    'Industry': industry_df,
    'Sea Level': sea_level_df,
    'Emitted': emitted_df,
    'Temp Change': temp_change_df,
    'Temperature': temperature_df
}

# Display dataframes to ensure data was loaded properly
for name, df in dfs.items():
    print(f"{name}")
    print(df.head())

Produces
  ghgYear                                       industryName  \
0  2010Q1                  Agriculture, Forestry and Fishing   
1  2010Q1                                       Construction   
2  2010Q1  Electricity, Gas, Steam and Air Conditioning S...   
3  2010Q1                                      Manufacturing   
4  2010Q1                                             Mining   

   totalco2concentration  
0               34175.16  
1                3919.65  
2               64387.33  
3               48376.39  
4               20071.35  
Continent
        continentName    code
0                 G20   NA120
1       Southern Asia    NASA
2        Eastern Asia   NA510
3            Americas  AMETMP
4  South-eastern Asia   NASEA
Country
          name continentName ISO3
0       Israel  Western Asia  ISR
1      Austria        Europe  AUT
2  Isle of Man        Europe  IMN
3   Seychelles        Africa  SYC
4      Somalia        Africa  SOM
GHG Emission
  ghgYear                    

In [38]:
# Perform EDA for each dataframe
def eda(df, df_name):
    print(f"EDA for {df_name}:\n")

    # Descriptive Statistics
    print("Descriptive Statistics:")
    print(df.describe(), '\n')
    
    # Check for missing values
    print("Missing Values:")
    print(df.isnull().sum(), '\n')

    # Analyze unique values in categorical columns
    for column_name, data_type in df.select_dtypes(include=['object', 'category']).items():
        unique_count = data_type.nunique()
        print(f"Unique values in {column_name}: {unique_count}")

    # Separator
    print("--------------------------")
        
for name, df in dfs.items():
    eda(df, name)

EDA for Produces:

Descriptive Statistics:
       totalco2concentration
count             540.000000
mean            49609.572796
std             68944.803789
min              3844.570000
25%             11827.437500
50%             23571.280000
75%             53007.375000
max            269678.490000 

Missing Values:
ghgYear                  0
industryName             0
totalco2concentration    0
dtype: int64 

Unique values in ghgYear: 54
Unique values in industryName: 10
--------------------------
EDA for Continent:

Descriptive Statistics:
       continentName   code
count             25     25
unique            25     25
top              G20  NA120
freq               1      1 

Missing Values:
continentName    0
code             0
dtype: int64 

Unique values in continentName: 25
Unique values in code: 25
--------------------------
EDA for Country:

Descriptive Statistics:
          name continentName ISO3
count      172           172  172
unique     172            17  172
top  